In [4]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import json as js
import ast as ast
import re
from textblob import TextBlob
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

ModuleNotFoundError: No module named 'textblob'

### Carga de JSON a DataFrames

In [2]:
games = pd.read_json('output_steam_games.json', lines = True)

In [3]:
list_rev = []
archivo1 = r'australian_user_reviews.json'
with open(archivo1, encoding='utf-8') as file:
    for line in file.readlines():
        list_rev.append(ast.literal_eval(line))

reviews = pd.DataFrame(list_rev)

In [4]:
list_items = []
archivo2 = r'australian_users_items.json'
with open(archivo2, encoding='utf-8') as file:
    for line in file.readlines():
        list_items.append(ast.literal_eval(line))

items = pd.DataFrame(list_items)

### Tratamiento de Datos

Desanidación de las tablas Reviews e Items.

In [5]:
reviews = reviews.explode('reviews', ignore_index=True)
reviews_expanded = pd.json_normalize(reviews['reviews'])
reviews_expanded=reviews_expanded.replace('',None)
reviews = reviews.join(reviews_expanded)
reviews.drop(columns=['reviews'],inplace=True)

In [6]:
items = items.explode('items', ignore_index=True)
items_expanded = pd.json_normalize(items['items'])
items_expanded = items_expanded.replace('', None)
items = items.join(items_expanded)
items.drop(columns=['items'], inplace=True)

Eliminación de filas y columnas con valores nulos o innecesarias para el análisis.

In [7]:
games = games.drop(columns=['publisher', 'title', 'url', 'reviews_url'])
games = games.dropna(how='all')

items = items.drop(columns=['steam_id', 'user_url'])
items = items.dropna()

reviews = reviews.drop(columns=['user_url', 'funny', 'last_edited', 'helpful'])
reviews = reviews.dropna(thresh=3)

Reemplazo de valores y Cambio de tipo de dato.

In [8]:
# PRICE:

games['price'] = games['price'].replace('Starting at $499.00', 499.00)
games['price'] = games['price'].replace('Starting at $449.00', 449.00)

mask = games['price'].apply(lambda x: isinstance(x, str))
games.loc[mask, 'price'] = 0.00

games['price'] = pd.to_numeric(games['price'], errors='coerce')


# FECHA:

def convertir_fecha(fecha):
    # Formatos conocidos
    for fmt in ['%B %Y', '%b %Y', '%Y-%m-%d', '%Y']:
        try:
            return pd.to_datetime(fecha, format=fmt)
        except ValueError:
            continue

    # Casos donde hay texto antes del año
    match = re.search(r'\b(\d{4})\b', fecha)
    if match:
        # Si encuentra un año, convertirlo
        return pd.to_datetime(match.group(1), format='%Y')

    return pd.NaT  # Si no coincide con ningún formato, devuelve NaT

# Aplicar la función de conversión
games['release_date'] = games['release_date'].apply(convertir_fecha)

# Extraer el año
games['release_date'] = games['release_date'].dt.year

# Cambio de nombre de la columna release_date
games = games.rename(columns={'release_date': 'year'})

# Cambio de nombre de la columna app_name
games = games.rename(columns={'app_name': 'item_name'})

# Cambio de nombre de la columna id
games = games.rename(columns={'id': 'item_id'})

# Valores nulos se rellenarán con cero para poder realizar el cambio de tipo de dato
games['item_id'] = games['item_id'].fillna(0)
# Cambio de tipo de dato de la columna item_id
games['item_id'] = games['item_id'].astype(int)

# Valores nulos se rellenarán con cero para poder realizar el cambio de tipo de dato
games['year'] = games['year'].fillna(0)
# Cambio de tipo de dato de la columna year
games['year'] = games['year'].astype(int)

Eliminación de filas duplicadas.

In [9]:
reviews = reviews.drop_duplicates()
items = items.drop_duplicates()

### Análisis de sentimiento con NLP

In [10]:
def analizar_sentimiento(row):
    reseña = row['review']
    recomendacion = row['recommend']
    
    # Caso: la reseña está vacía
    if pd.isnull(reseña) or reseña.strip() == '':
        # Si no hay reseña, usar la recomendación para determinar el sentimiento
        if recomendacion:
            return 2  # Si recomienda el juego, asignar positivo
        else:
            return 0  # Si no recomienda el juego, asignar negativo
    
    # Si la reseña está presente, hacer el análisis de sentimiento
    analysis = TextBlob(reseña).sentiment.polarity
    
    # Asignar el sentimiento basado en la polaridad
    if analysis < -0.1:
        return 0  # Sentimiento negativo
    elif analysis > 0.1:
        return 2  # Sentimiento positivo
    else:
        return 1  # Neutral

# Aplicar la función al dataset
reviews['sentiment_analysis'] = reviews.apply(analizar_sentimiento, axis=1)

### Guardar los archivos limpiados

In [11]:
# Ruta especifica donde quiero guardar los archivos JSON
ruta_games = 'C:\\Users\\Sarita\\Desktop\\SOY HENRY\\Proyecto 1\\games_cleaned.json'
ruta_reviews = 'C:\\Users\\Sarita\\Desktop\\SOY HENRY\\Proyecto 1\\reviews_cleaned.json'
ruta_items = 'C:\\Users\\Sarita\\Desktop\\SOY HENRY\\Proyecto 1\\items_cleaned.json'

# Guarda el DataFrame en formato JSON en la ruta especificada
games.to_json(ruta_games, orient='records', lines=True)
reviews.to_json(ruta_reviews, orient='records', lines=True)
items.to_json(ruta_items, orient='records', lines=True)

In [12]:
games_cleaned = pd.read_json('games_cleaned.json', lines = True)
reviews_cleaned = pd.read_json('reviews_cleaned.json', lines = True)
items_cleaned = pd.read_json('items_cleaned.json', lines = True)

### FastAPI

Consulta 1: Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora.

In [13]:
# Función para calcular la cantidad de items y el porcentaje de contenido gratuito
def developer_items_free(df):

    df_filtrado = games_cleaned[games_cleaned['developer'] == df]
    games_agrupado = df_filtrado.groupby(games_cleaned['year']).agg(
    cantidad_items=('item_name', 'count'),
    items_gratuitos=('price', lambda x: (x == 0).sum())
    )

    # Calculamos el porcentaje de apps gratuitas
    games_agrupado['contenido_free'] = (games_agrupado['items_gratuitos'] / games_agrupado['cantidad_items']) * 100
    games_agrupado['contenido_free'] = games_agrupado['contenido_free'].round(2).astype(str) + '%'

    # Reseteamos el índice para tener una estructura de DataFrame más convencional
    games_agrupado = games_agrupado.reset_index()
    return (games_agrupado)

In [1]:
r = developer_items_free('SmiteWorks USA, LLC')
print(r)

NameError: name 'developer_items_free' is not defined

Consulta 2: Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [15]:
# Función para calcular cantidad de dinero gastado por el usuario, el porcentaje de recomendación y cantidad de items.
def user_statistics(user):

    # Filtrar los juegos comprados por el usuario
    user_items_data = items_cleaned[items_cleaned['user_id'] == user]
    # Filtrar las recomendaciones hechas por el usuario
    user_reviews_data = reviews_cleaned[reviews_cleaned['user_id'] == user]
    # Unir el DataFrame de games con el de items
    user_data = pd.merge(games_cleaned, user_items_data, on=['item_name'], how='right')

    # Calcular la cantidad total de dinero gastado
    total_spent = user_data['price'].sum()
    total_spent = round(total_spent, 2)

    # Calcular la cantidad total de juegos comprados
    total_games = user_items_data.shape[0]

    # Calcular el porcentaje de recomendación (True vs False en 'recommend')
    total_recommendations = user_reviews_data['recommend'].sum()  # Cuenta cuántos son True
    recommendation_percentage = (total_recommendations / total_games) * 100 if total_games > 0 else 0

    # Redondear el porcentaje a 2 decimales
    recommendation_percentage = round(recommendation_percentage, 2)

    # Devolver los resultados en un diccionario
    return {
        "user_id": user,
        "total_spent": f"${total_spent}",
        "recommendation_percentage": f"{recommendation_percentage}%",
        "total_games": total_games
    }


In [ ]:
s = user_statistics('evcentric')
print(s)

Consulta 3: Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.

In [17]:
# Función para calcular el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
def genre(genero):

    # Filtrar los juegos que contengan el genero dado
    df_juegos = games_cleaned[games_cleaned['genres'].apply(lambda x: isinstance(x, list) and genero in x)]
   
    # Unir el DataFrame de juegos con el de items
    df_merged = pd.merge(items_cleaned, df_juegos, on='item_name', how='inner')

    # Agrupar por usuario y sumar las horas jugadas
    df_grouped = df_merged.groupby('user_id').agg(total_playtime=('playtime_forever', 'sum'))

    # Ordenar por el total de horas jugadas en orden descendente
    df_grouped = df_grouped.sort_values(by='total_playtime', ascending=False).reset_index()

    # Guardar en una variable el usuario con más horas jugadas
    most_played_user = df_grouped['user_id'][0]  # Usuario con más horas jugadas

    # Filtrar el DataFrame para obtener solo los datos del usuario específico
    df_usuario = df_merged[df_merged['user_id'] == most_played_user]

    # Agrupar por año y sumar las horas jugadas del usuario
    df_grouped_usuario = df_usuario.groupby('year').agg(total_playtime=('playtime_forever', 'sum')).reset_index()

    return (f'Usuario con más horas jugadas para {genero}: {most_played_user}', df_grouped_usuario)

In [ ]:
t = genre('Action')
print(t)

Consulta 4: Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

In [19]:
# Función para obtener el top 3 de desarrolladores con juegos más recomendados en un año dado
def best_developer(año):

    # Filtrar los juegos lanzados en el año dado
    juegos_del_año = games_cleaned[games_cleaned['year'] == año]

    # Unir los DataFrames de games y reviews en base al nombre del juego (app_name)
    merged_df = pd.merge(juegos_del_año, reviews_cleaned, on='item_id', how='inner')

    # Filtrar por recomendaciones positivas y comentarios positivos
    juegos_recomendados = merged_df[(merged_df['recommend'] == True) & (merged_df['sentiment_analysis'] == 2)]

    # Agrupar por desarrollador y contar las recomendaciones positivas
    desarrollador_recomendaciones = juegos_recomendados.groupby('developer').size().reset_index(name='num_recommendations')

    # Ordenar los desarrolladores por la cantidad de recomendaciones y seleccionar el top 3
    top_3_desarrolladores = desarrollador_recomendaciones.sort_values(by='num_recommendations', ascending=False).head(3)

    return top_3_desarrolladores

In [ ]:
u = best_developer(2017)
print(u)

Consulta 5: Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [21]:
# Función para devolver la cantidad de reseñas positivas y negativas de una desarrolladora
def developer_reviews(desarrolladora):

    # Filtrar los juegos que fueron desarrollados por la desarrolladora dada
    juegos_de_desarrolladora = games_cleaned[games_cleaned['developer'] == desarrolladora]

    # Unir los DataFrames de juegos y reseñas en base al id del juego (item_id)
    merged_df = pd.merge(juegos_de_desarrolladora, reviews_cleaned, on='item_id', how='inner')

    # Contar la cantidad de reseñas positivas y negativas
    reseñas_positivas = merged_df[merged_df['sentiment_analysis'] == 2].shape[0]
    reseñas_negativas = merged_df[merged_df['sentiment_analysis'] == 0].shape[0]

    # Devolver el resultado en formato diccionario
    resultado = {
        desarrolladora: {
            "Positive": reseñas_positivas,
            "Negative": reseñas_negativas
        }
    }

    return resultado

In [ ]:
v = developer_reviews('Freejam')
print(v)

### Modelo de Recomendación

In [23]:
games1 = games_cleaned.copy()

In [24]:
games1['tags_combined'] = games1['tags'].fillna('').apply(lambda x: ' '.join(x))

In [25]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(games1['tags_combined'])

In [26]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [27]:
def year_penalty(year1, year2):
    return 1 - (abs(year1 - year2) / 10)

In [28]:
def get_recommendations(item_id, num_recommendations=5):
    try:
        # Obtener el índice del juego
        idx = games1[games1['item_id'] == item_id].index[0]
        
        # Obtener el año del juego
        target_year = games1.loc[idx, 'year']

        # Obtener las similitudes basadas en géneros
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Ajustar las similitudes en base a la cercanía de los años
        for i, (index, score) in enumerate(sim_scores):
            game_year = games1.loc[index, 'year']
            sim_scores[i] = (index, score * year_penalty(target_year, game_year))

        # Ordenar por similitud y obtener los 5 juegos más similares
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:num_recommendations+1]

        # Obtener los índices de los juegos recomendados
        game_indices = [i[0] for i in sim_scores]
        
        # Devolver los juegos recomendados (IDs y nombres)
        return games1[['item_id', 'item_name']].iloc[game_indices]

    except IndexError:
        return "Juego no encontrado"

In [ ]:
l = get_recommendations(200870, num_recommendations=5)
print(l)

In [ ]:
games1[games1['item_id']==331450]

In [ ]:
games1[games1['item_name']=="Assassin's Creed Revelations - The Ancestors Character Pack"]